In [2]:
import re

class Dir:    
    def __init__(self, path, parent=None, children=None, total_disk_size=70000000):
        self.path = path
        self.parent = parent
        self.children = children or []
        self.total_disk_size = total_disk_size

    def get_size(self):
        if not self.children:
            return 0
        
        size = 0
        for child in self.children:
            if isinstance(child, File):
                size += child.size
            elif isinstance(child, Dir):
                size += child.get_size()
        return size
    
    def get_dirs_smaller_than(self, dirs, limit=100000):
        if not self.children:
            return []
        
        size = 0
        for child in self.children:
            if isinstance(child, Dir):
                size = child.get_size()
                if size <= limit:
                    dirs.append(child)
                    
                child.get_dirs_smaller_than(dirs, limit)
                
        return dirs
    
    def cd(self, path):
        if path == '..':
            return self.parent
        
        if self.children:
            matches = [c for c in self.children if c.path == path and isinstance(c, Dir)]
            if len(matches) != 1:
                raise Exception(f'Unexpectedly there was not exactly 1 match {path} {matches}')
            return matches[0]

        raise Exception(f'No adjacent folder with path {path}')
        
    def add(self, node):
        matches = [c for c in self.children if c.path == node.path and type(node) == type(c)]
        if matches:
            return None
        self.children.append(node)
        
    def get_child_dir_to_delete(self, desired_free_space):
        dirs = sorted([(d.path, d.get_size()) for d in self.get_dirs_smaller_than([], limit=10000000000)], key=lambda k: k[1])

        to_free = desired_free_space - (self.total_disk_size - root.get_size())
        for path, size in dirs:
            if size > to_free:
                return path, size
                break
                
        return None

    def __str__(self, level=0):
        ret = "  "*level+'dir '+repr(self.path)+"\n\n"
        for child in self.children:
            ret += child.__str__(level+1)
        return ret
    

class File:
    def __init__(self, path: str, size: int, parent):
        self.path = path
        self.size = size
        self.dir = parent
        
    def __str__(self, level):
        return "  "*level + f'{self.size} {self.path}\n\n'
        
with open('../inputs/7.txt') as f:
    root = None
    cwd = None
    counter = 0
    for line in f:
        if line.startswith('$ cd'):
            
            result = re.search(r"\$ cd (.*)", line)
            to = result.groups()[0]
            from_ = cwd.path if cwd else '/'
            if not cwd:
                root = Dir(to)
                cwd = root
            elif to == '/':
                cwd = root
            else:
                cwd = cwd.cd(to)
        elif line.startswith('dir'):
            result = re.search(r"dir (.*)", line)
            d = result.groups()[0]
            cwd.add(Dir(d, parent=cwd))
        elif line.startswith('$ ls'):
            continue
        else:
            result = re.search(r"([0-9]+) (.*)", line)
            size, f = result.groups()
            cwd.add(File(f, size=int(size), parent=cwd))
            
        counter += 1

print(sum([d.get_size() for d in root.get_dirs_smaller_than([])]))
print(root.get_child_dir_to_delete(30000000))

1648397
('nbq', 1815525)
